In [4]:
%pip install statsforecast
%pip install holidays
%pip install pandas


[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip



Note: you may need to restart the kernel to use updated packages.
     -------------------------------------- 922.7/922.7 kB 4.9 MB/s eta 0:00:00



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.2.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
# from fourier_forecast.fourier_forecast import FourierForecast
from statsforecast.models import AutoARIMA, ARIMA
from holidays import country_holidays
from datetime import date, timedelta
from numpy.typing import NDArray
from itertools import product
# from prophet import Prophet
import pandas as pd
import numpy as np

In [6]:
from arima import Model

In [28]:
def find_holidays(start_date: date, end_date: date) -> list[date]:
    """return dictionary with just holiday dates by market"""
    yrs = range(start_date.year, end_date.year + 1)
    return list(country_holidays(country="US", years=yrs).keys())


def add_us_holidays(df: pd.DataFrame) -> pd.DataFrame:
    """return dataframe by date & market with is_holiday column where True for holiday date and false otherwise"""
    h = find_holidays(df["ds"].min(), df["ds"].max())
    return df.assign(is_holiday=df["ds"].apply(lambda z: z in h))


In [ ]:
def calculate_mape(predictions: NDArray, actuals: NDArray) -> float:
    return np.abs(predictions / actuals - 1).mean()

[Data](https://data.cityofevanston.org/dataset/CTA-Ridership-Daily-Boarding-Totals/bnrf-isry)

In [30]:
df = (pd.read_csv("./cta_ridership_totals.csv")
        .assign(ds=lambda x: pd.to_datetime(x["date"]).dt.date,
                y=lambda x: x["total_rides"]
                )
        .sort_values(by="ds", ascending=True)
)
df = add_us_holidays(df)
df = df[["ds", "y", "is_holiday"]]
display(df)

,ds,y,is_holiday
0,2001-01-01,423647,True
1,2001-01-02,1282779,False
2,2001-01-03,1361355,False
3,2001-01-04,1420032,False
4,2001-01-05,1448343,False
...,...,...,...
6508,2018-10-27,928367,False
6509,2018-10-28,649725,False
6510,2018-10-29,1601855,False
6511,2018-10-30,1636184,False


In [31]:
train_years = 2
first_year, last_year = 2003, 2018

In [32]:
ff_results = np.zeros(last_year - first_year + 1, dtype=np.float64)
prophet_results = np.zeros(last_year - first_year + 1, dtype=np.float64)
aa_results = np.zeros(last_year - first_year + 1, dtype=np.float64)

In [33]:
SplitType = tuple[pd.DataFrame, pd.DataFrame, pd.DataFrame, pd.DataFrame]

def train_test_split(test_year: int, data: pd.DataFrame) -> SplitType:
        

        train_dates = (date(test_year - train_years, 1, 1), date(test_year - 1, 12, 31))
        test_dates = (date(test_year, 1, 1), date(test_year, 12, 31))

        train = df[df["ds"].isin(train_dates)]
        test = df[df["ds"].isin(test_dates)]

        return train.drop("y"), test.drop("y"), train["y"], test["y"]

In [ ]:
for i, test_year in enumerate(range(first_year, last_year + 1)):

    train_dates = (date(test_year - train_years, 1, 1), date(test_year - 1, 12, 31))
    test_dates = (date(test_year, 1, 1), date(test_year, 12, 31))

    train_index = df[df["ds"].isin(train_dates)].index
    test_index = df[df["ds"].isin(test_dates)].index
    h = test_index.size

    train_df = df[df.index.isin(train_index)]
    test_df = df[df.index.isin(test_index)]
    train_hols = hols[train_index]
    test_hols = hols[test_index]

    # ff = FourierForecast(weekly_seasonality_terms=3, yearly_seasonality_terms=10, log_y=True)
    # ff.fit(train_df["y"].values, regressors=train_hols)

    # preds = ff.predict(h=h, regressors=test_hols)
    # ff_results[i] = calculate_mape(preds, test_df["y"].values)

    m = Prophet(seasonality_mode="multiplicative", uncertainty_samples=False)
    m.add_country_holidays(country_name="US")
    m.fit(train_df)

    future_df = m.make_future_dataframe(periods=h).tail(h)
    pred_df = m.predict(future_df)

    prophet_results[i] = calculate_mape(pred_df["yhat"].values, test_df["y"].values)


    aa = ARIMA(season_length=[7])
    aa.fit(train_df["y"].values, train_hols.astype(np.float64))
    aa_preds = aa.predict(h, test_hols.astype(np.float64), level=None)["mean"]
    aa_results[i] = calculate_mape(aa_preds, test_df["y"].values)

print("FourierForecast:", ff_results)
print("FourierForecast:", ff_results.mean().round(3))
print("Prophet:", prophet_results)
print("Prophet:", prophet_results.mean().round(3))
# print("AutoArima:", aa_results)
# print("AutoArima:", aa_results.mean().round(3))

In [3]:
model = Model.arima(1, 1, 1)

In [4]:
model.coefs_fit


AttributeError: 'arima.Model' object has no attribute 'coefs_fit'